## Sources:

- https://towardsdatascience.com/how-to-train-bert-for-q-a-in-any-language-63b62c780014

In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering, BertTokenizerFast, AdamW
from datasets import load_dataset
import wandb
from tqdm.notebook import tqdm, trange, tqdm_notebook # show progress bar
from sklearn.metrics import f1_score

## Load Dataset and Split into Training and Validation Datasets

In [2]:
dataset = load_dataset("squad")
train_dataset, validation_dataset = dataset['train'], dataset['validation']
train_dataset

Reusing dataset squad (/home/christian/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 280.58it/s]


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

## See a sample question answer set

In [3]:
def add_end_idx(answers, contexts):
    new_answers = []
    # loop through each answer-context pair
    for answer, context in tqdm(zip(answers, contexts)):
        # quick reformating to remove lists
        answer['text'] = answer['text'][0]
        answer['answer_start'] = answer['answer_start'][0]
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
        new_answers.append(answer)
    return new_answers

def prep_data(dataset):
    questions = dataset['question']
    contexts = dataset['context']
    answers = add_end_idx(
        dataset['answers'],
        contexts
    )
    return {
        'question': questions,
        'context': contexts,
        'answers': answers
    }

## Initialize Model, Tokenizer and Optimizer

In [4]:
# train_dataset = prep_data(train_dataset)
# validation_dataset = prep_data(validation_dataset)
# train_dataset['answers'][:3]

In [5]:
pretrained_model_name: str = 'bert-large-uncased-whole-word-masking-finetuned-squad'
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# tokenize
# train = tokenizer(train_dataset['context'], train_dataset['question'],
#                           truncation=True, padding='max_length',
#                           max_length=512, return_tensors='pt')

# validation = tokenizer(validation_dataset['context'], validation_dataset['question'],
#                           truncation=True, padding='max_length',
#                           max_length=512, return_tensors='pt')
            

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [6]:
# tokenizer.decode(train['input_ids'][0])[:855]
# tokenizer.decode(validation['input_ids'][0])[:855]

In [7]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in tqdm(range(len(answers))):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


In [8]:
# apply function to the data
# add_token_positions(train, train_dataset['answers'])
# add_token_positions(validation, validation_dataset['answers'])

### This function adds two more tensors to the `Encoding` object (which will be fed into Bert)

- the `start_positions` and
- the `end_positions` tensors.


In [9]:
# train.keys()

In [10]:
# train['start_positions'][:5], train['end_positions'][:5]

### Training

For the training, Pytorch is used and the dataset will be converted to a Pytorch Dataset.

In [11]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# train_dataset = SquadDataset(train)
# validation_dataset = SquadDataset(validation)

In [12]:
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# len(train_loader)

In [13]:
# validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
# len(validation_loader)

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min', 
    threshold=0.001, 
    cooldown=0, 
    eps=1e-07, 
    patience=2, 
    verbose=True, 
    min_lr=1e-5, 
    factor=0.5)

In [15]:
def get_next_dataloader(start_index: int, end_index: int, shuffle: bool = True, dataset_key: str = 'train') -> DataLoader:
    part_dataset = dataset[dataset_key][start_index:end_index]
    part_dataset = prep_data(part_dataset)
    part_train = tokenizer(part_dataset['context'], part_dataset['question'],
                          truncation=True, padding='max_length',
                          max_length=512, return_tensors='pt')
    tokenizer.decode(part_train['input_ids'][0])[:855]
    # apply function to the data
    add_token_positions(part_train, part_dataset['answers'])
    part_dataset = SquadDataset(part_train)
    return DataLoader(part_dataset, batch_size=32, shuffle=shuffle)


In [16]:
wandb.init(
    project='BERT Training',
    config={
        'batch_size': 32,
        'dataset': 'SQUAD 2.0'
    })

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: my-god-its-full-of-stars (use `wandb login --relogin` to force relogin)


In [ ]:
def convert_all_input_ids_to_tokens(input_ids):
    return [tokenizer.convert_ids_to_tokens(ids) for ids in input_ids]

#### Initialize Training Preliminaries

In [ ]:
max_train_batch_length: int = 2738
max_validation_batch_length: int = 331

num_episodes: int = 100
save_zones = [epoch for epoch in range(10, 101, 25)]
step = 5

training_losses = []
validation_losses = []

In [ ]:
for epoch in trange(num_episodes):

    model.train()
    train_accuracy = 0
    train_total = 0

    f1_start = 0
    f1_end = 0


    for start_range in trange(0, max_train_batch_length, step):
        end_range = min(max_train_batch_length, start_range + step)
        loop = tqdm_notebook(get_next_dataloader(start_range, end_range), desc=f'<{start_range}:{end_range}>')
        for batch in loop:
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask,
                            start_positions=start_positions,
                            end_positions=end_positions)
            
 
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
            answer_start = torch.argmax(start_scores, dim=1)
            answer_end = torch.argmax(end_scores, dim=1)

            train_accuracy += (start_positions == answer_start).sum().float()
            train_accuracy += (end_positions == answer_end).sum().float()

            # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
            f1_start += f1_score(start_positions.cpu(), answer_start.cpu(), average='weighted', zero_division=0)
            f1_end += f1_score(end_positions.cpu(), answer_end.cpu(), average='weighted', zero_division=0)

            train_total += len(start_positions)
        
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            loop.set_description(f'Epoch {epoch + 1} Training <{start_range}:{end_range}>')
            loop.set_postfix(loss=loss.item())

    if epoch in save_zones:
        torch.save(model.state_dict(), f'model/bert_model_{epoch}.pt')
        torch.save(optimizer.state_dict(), f'model/optmizer_{epoch}.pt')
        torch.save(scheduler.state_dict(), f'model/scheduler_{epoch}.pt')

    training_losses.append(loss.item())
    wandb.log({
        'training loss': loss.item(),
        'training accuracy': train_accuracy / (train_total * 2),
        'training f1 score': (f1_start + f1_end) / (train_total * 2)
    })

    validation_accuracy = 0
    validation_total = 0

    f1_start = 0
    f1_end = 0

    for start_range in trange(0, max_validation_batch_length, step):
        end_range = min(max_validation_batch_length, start_range + step)
        loop = tqdm(get_next_dataloader(start_range, end_range, shuffle=False, dataset_key='validation'))
        

        with torch.no_grad():
            model.eval()
            for batch in loop:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                start_positions = batch['start_positions'].to(device)
                end_positions = batch['end_positions'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask,
                                start_positions=start_positions,
                                end_positions=end_positions)

                start_scores = outputs.start_logits
                end_scores = outputs.end_logits
                answer_start = torch.argmax(start_scores, dim=1)
                answer_end = torch.argmax(end_scores, dim=1)

                validation_accuracy += (start_positions == answer_start).sum().float()
                validation_accuracy += (end_positions == answer_end).sum().float()

                f1_start += f1_score(start_positions.cpu(), answer_start.cpu(), average='weighted', zero_division=0)
                f1_end += f1_score(end_positions.cpu(), answer_end.cpu(), average='weighted', zero_division=0)

                validation_total += len(start_positions)
                
                loss = outputs.loss

                loop.set_description(f'Epoch {epoch + 1} Validation <{start_range}:{end_range}>')
                loop.set_postfix(loss=loss.item())

                scheduler.step(loss)

    validation_losses.append(loss.item())
    wandb.log({
        'validation loss': loss.item(), 
        'validation accuracy': validation_accuracy / validation_total,
        'validation f1 score': (f1_start + f1_end) / (validation_total * 2)
    })

#### Save Final State of Trained Model

In [ ]:
torch.save(model.state_dict(), f'model/bert_model_{epoch}.pt')
torch.save(optimizer.state_dict(), f'model/optmizer_{epoch}.pt')
torch.save(scheduler.state_dict(), f'model/scheduler_{epoch}.pt')